'''
v1: Scienceon, DBPIA, NTIS AuthorPapers 접근하여 Rawdata에서 필터링된 papers를 추가
v2: Answer 변경 및 f_id 수정 중
v3: time, tqdm 추가하여 속도 확인
v4: Scienceon, DBPIA에 mng로 검색 추가
v5: DBPIA 삭제하고 KCI 추가 & Rawdata 먼저 접급으로 변경, answer check 추가
v5.1: 대학테이블, 정답비교 코드 추가
v6: nFilter, pFilter 조건 비교 함수 추가(ntis_filter, paper_filter)
v6.1: 대학테이블, 정답비교 코드 추가, sites => site, Answer['inst'] 추가
v7: Autor 접근 삭제, Rawdata에서 소속 이름 가져오기 추가, inst값 없는 문제 해결, A_id 나눠들어가는 문제 해결
v8: 종훈이 코드 불러오기, 웹 필터 카테고리
'''

In [11]:
from pymongo import MongoClient
import itertools
from itertools import islice
import numpy as np
import jaro
import time
import multicpu

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근
filters_category = client['PUBLIC']['FilterCategory']
f_id = 0 #input
keyid = 675 #keyid

fid_key_query = filter_info.find_one({ '$and': [{ 'fId': f_id }, { 'keyId': keyid }]}) #f_id serach
ninst = []
nrsc = []
nfund = []
nyear = []
pinst = []
pyear = []
pjournal = []
plang = []

if  fid_key_query != None: #f_id check
    for key in fid_key_query.keys() :
        if key == 'nFilter':
            ninst = fid_key_query[key]['inst'] #소속
            nrsc = fid_key_query[key]['rsc'] #공동저자수
            nfund = fid_key_query[key]['fund'] #과제수주비
            nyear = fid_key_query[key]['year'] #연도

        elif key == 'pFilter' :
            pinst = fid_key_query[key]['inst']
            pyear = fid_key_query[key]['year']
            pjournal = fid_key_query[key]['journal']
            plang = fid_key_query[key]['lang']

scion_aut = client['SCIENCEON']['Author']
ntis_aut = client['NTIS']['Author']
kci_aut = client['KCI']['Author']

scion_raw = client['SCIENCEON']['Rawdata']
ntis_raw = client['NTIS']['Rawdata']
kci_raw = client['KCI']['Rawdata']

scion_key_query = scion_raw.find({ 'keyId' : keyid })
ntis_key_query = ntis_raw.find({ 'keyId' : keyid })
kci_key_query = kci_raw.find({ 'keyId' : keyid })

auts = [scion_aut, ntis_aut, kci_aut] #Author
key_querys = [scion_key_query, ntis_key_query, kci_key_query] #Rawdata
id_domestic = client['ID']['Domestic'] #Domestic

mng_id = [] # Author id
paper = []
Answer_dict = {} # Answer result
site = ['SCIENCEON', 'NTIS', 'KCI']
fund = [0, 50000000, 100000000, 300000000, 500000000, 1000000000, 10000000000000 ]
rsc = [0, 10, 30, 50, 100, 100000]
savetime1 = 0
savetime2 = 0

f_nyear = {}
f_ninst = {}
f_nfund = {'0':0, '1':0, '2':0, '3':0, '4':0, '5':0}
f_nrsc = {'0':0, '1':0, '2':0, '3':0, '4':0}
f_pyear = {}
f_pinst = {}
f_pjournal = {}
f_plang = {}

def simple_filter(value, filters) :
    if value in filters or filters == []:
        return True
    return False
        
def complex_filter(value, filters, base) :
    if filters == [] :
        return True

    for j in range(len(filters)):
        if base[filters[j]] <= float(value) < base[filters[j]+1]:
            return True
    return False

def fc_simple_filter(category, fc_dict):
    if category not in fc_dict:
        fc_dict[category] = 1
    fc_dict[category] += 1
    return fc_dict

def fc_complex_filter(category, base, fc_dict):
    for j in range(len(base)-1):
        if base[j] <= float(category) < base[j+1]:
            fc_dict[str(j)] += 1
            return fc_dict

for i in range(len(key_querys)):
    mng_dict = {}
    start1 = time.time()
    for key_query in key_querys[i]: #rawdata(magid, paper) insert
        if site[i] == 'NTIS' :
            ori_inst = key_query['originalName']
            ntis_rsc = int(key_query['cntRscMan']) + int(key_query['cntRscWom'])
            ntis_fund = key_query['totalFund']
            ntis_year = key_query['prdStart'][:4]
            exi_inst = key_query['ldAgency']
            mng_name =  key_query['mng']
            mng_id = key_query['mngId']
            paper = key_query['_id']

            if simple_filter(ori_inst, ninst) and simple_filter(ntis_year, nyear) and complex_filter(ntis_fund, nfund, fund) and complex_filter(ntis_rsc, nrsc, rsc):
                if mng_id not in mng_dict:
                    mng_dict[mng_id] = {'name' : mng_name, 'inst' : exi_inst, 'papers' : [], 'oriInst' : ori_inst}
                mng_dict[mng_id]['papers'].append(paper)
                f_nyear = fc_simple_filter(ntis_year, f_nyear)
                f_ninst = fc_simple_filter(ori_inst, f_ninst)
                f_nfund = fc_complex_filter(ntis_fund, fund, f_nfund)
                f_nrsc = fc_complex_filter(ntis_rsc, rsc, f_nrsc)

        else:
            paper_year =  key_query['issue_year'][:4]
            paper_journal = key_query['journal']
            ori_inst = key_query['originalName'].split(';')[-2]
            paper_lang = key_query['issue_lang']
            exi_inst = key_query['author_inst'].split(';')[-2]
            mng_name = key_query['author'].split(';')[-2]
            mng_id = key_query['mngId']
            paper = key_query['_id']

            if simple_filter(paper_year, pyear) and simple_filter(paper_journal, pjournal) and simple_filter(ori_inst, pinst) and simple_filter(paper_lang, plang):
                if mng_id not in mng_dict:
                    mng_dict[mng_id] = {'name' : mng_name, 'inst' : exi_inst, 'papers' : [], 'oriInst' : ori_inst}
                mng_dict[mng_id]['papers'].append(paper)
                f_pyear = fc_simple_filter(paper_year, f_pyear)
                f_pinst = fc_simple_filter(ori_inst, f_pinst)
                f_pjournal = fc_simple_filter(paper_journal.replace(".", "^"), f_pjournal)
                f_plang = fc_simple_filter(paper_lang, f_plang)
                
    end1 = time.time()
    print(site[i], end1-start1)
    savetime1 += end1-start1

    for mng_one in mng_dict :
        oriinst = mng_dict[mng_one]['oriInst']
        exiinst = mng_dict[mng_one]['inst']
        mng_name = mng_dict[mng_one]['name']
        paper = mng_dict[mng_one]['papers']

        Answer = {'fid': f_id, 'keyId': keyid, 'name' : mng_name , 'inst': oriinst, site[i] : {'inst' :exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst} }
        
        if mng_name not in Answer_dict and mng_name+'_0' not in Answer_dict : #동명이인이 없을 때
            Answer_dict[mng_name] = Answer
        else :
            count = 0
            flag = True
            while flag :
                temp = None 
                tempName = mng_name
                
                if tempName in Answer_dict : # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = mng_name+'_'+str(count) # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                        
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' and key != 'keyId' and key != 'fid' and key != 'inst': 
                        src = ""
                        tgt = ""

                        if len(exiinst) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = exiinst

                        elif len(exiinst) < len(temp[key]['inst']):
                            src = exiinst
                            tgt = temp[key]['inst']

                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == exiinst or (src != "" and src in tgt) :  # 소속 같을때
                                Answer_dict[tempName][site[i]]['A_id'].extend([mng_one])
                                Answer_dict[tempName][site[i]]['papers'].extend(paper)
                                flag = False
                                break

                            elif mng_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[mng_name+'_'+str(count+1)] = Answer

                                if tempName == mng_name:
                                    Answer_dict[mng_name+'_0'] = temp
                                    del Answer_dict[mng_name]

                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == exiinst  or (src != "" and src in tgt):  # 소속 같을때
                                Answer_dict[tempName][site[i]] =  {'inst' : exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst}
                                if '대학교' in Answer_dict[tempName][site[i]]['oriInst'] and '대학교' not in Answer_dict[tempName]['inst']:
                                    Answer_dict[tempName]['inst'] = Answer_dict[tempName][site[i]]['oriInst']
                                flag = False
                                break
                            
                            elif mng_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[mng_name+'_'+str(count+1)] = Answer

                                if tempName == mng_name:
                                    Answer_dict[mng_name+'_0'] = temp
                                    del Answer_dict[mng_name]

                count += 1
    end2 = time.time()
    savetime2 += end2-end1

# print(f_nyear)
# print(f_ninst)
# print(f_nfund)
# print(f_nrsc)
# print(f_pyear)
# print(f_pinst)
# print(f_pjournal)
# print(f_plang)

filter_dict= {'keyId': keyid, 'fId': f_id, 'paper': { 
                'year': {'list': f_pyear, 'k': 'year', 'v': '연도' },
                'inst': {'k': 'inst', 'list': f_pinst, 'v': '소속', 'f': 'false' },
                'journal': {'list': f_pjournal, 'k': 'journal', 'v': '저널'},
                'lang': {'list': f_plang, 'k': 'lang', 'v': '언어' }
            },
            'project': {
                'year': {'list': f_nyear, 'k': 'year', 'v': '연도' },
                'inst': {'list': f_ninst, 'k': 'inst', 'v': '소속' },
                'fund': {'k': 'fund', 'v': '과제수주비', 'list': f_nfund },
                'rsc': {'k': 'rsc', 'v': '참여인원', 'list': f_nrsc }
            }} 
print(filter_dict)
# filters_category.insert_one(filter_dict)
# print(savetime1, savetime2)
# print(sorted(Answer_dict.items()))

SCIENCEON 0.010998964309692383
NTIS 0.008970260620117188
KCI 0.07280659675598145
{'keyId': 675, 'fId': 0, 'paper': {'year': {'list': {'2012': 2, '2013': 3, '2014': 2, '2015': 3, '2017': 3, '2021': 205, '2016': 3, '2019': 3, '2020': 3}, 'k': 'year', 'v': '연도'}, 'inst': {'k': 'inst', 'list': {'서울과학기술대학교': 4, '한국항공대학교': 2, '연세대학교': 9, '전주대학교': 5, '전자부품연구원 실감정보플랫폼연구센터': 2, '미디어네트워킹연구실': 2, '(주)인스페이스': 2, '국민대학교': 5, '한세대학교': 4, '한남대학교': 3, '숭실대학교': 4, '인하대학교': 5, '세종대학교': 3, 'DEPARTMEN': 2, '울산대학교': 3, '광운대학교': 9, '중앙대학교': 7, '강남대학교': 3, '삼육대학교': 3, 'TEELABS': 2, '오모션': 2, '한국전자기술연구원': 2, '상명대학교': 6, '서원대학교': 3, '한국전자통신연구원': 3, '남서울대학교': 2, '부산가톨릭대학교': 3, '배재대학교': 3, '동의대학교': 3, '동국대학교': 7, '수원대학교': 2, '공주대학교': 2, '홍익대학교': 9, '한성대학교': 4, '동서대학교': 2, '장로회신학대학교': 3, '컬쳐커넥션': 2, '경희대학교': 5, '수원여자대학교': 3, '고려대학교': 6, '부산대학교': 4, '한양대학교': 5, '호서대학교': 3, 'KC대학교': 2, '한국과학기술정보연구원': 2, '협성대학교': 2, '서울교육대학교': 4, '전남대학교': 6, '한국음악콘텐츠협회': 2, '충남대학교': 2, '목포대학교': 2, '안동대학교': 3, '한국건설기술연구원': 2, '조선대학교'

In [10]:
def filter(site, rawdata):
    if site == 'NTIS' :
        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""

    else :
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        
        if rawdata['paper_keyword'] == [] or rawdata['paper_keyword'] == 'None':
            keyword = []
        elif len(rawdata['paper_keyword']) > 1:
            for i in range(0, len(rawdata['paper_keyword'])):
                keyword = []
                keyword.append(rawdata['paper_keyword'][i].replace(" ", "").split("."))            
        else:
            keyword = rawdata['paper_keyword'].replace(" ", "").split(".")

        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']

    return coauthor, year, keyword, journal, conference, title

def Secondary_filter(name, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst = 0
    weight = 0
    joc = 0
    coauthor1, year1, keyword1, journal1, conference1, title1 = filter(site1, raw_one1)
    coauthor2, year2, keyword2, journal2, conference2, title2 = filter(site2, raw_one2)

    if inst1 == inst2:
        inst = 1
    else:
        inst = jaro.jaro_winkler_metric(inst1, inst2)

    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2 or inst >= 0.8: #or mng1 == mng2:
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else 0
    else:
        if inst >= 0.8:
            weight = 3
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    #print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [14]:
raw_dbs = {'NTIS' : ntis_raw, 'SCIENCEON' : scion_raw, 'KCI' : kci_raw}
savetime1 = 0
savetime2 = 0
def getRaw(name):
    if 'raws' not in Answer_dict[name]:
        raws = []
        for site_one in site:
            if site_one in Answer_dict[name]:
                for c in raw_dbs[site_one].find({"_id": {"$in": Answer_dict[name][site_one]['papers']}}):
                    c['site'] = site_one
                    raws.append(c)
        
        Answer_dict[name]['raws'] = raws

processedList = []
deleteList = []

for Answer_one in Answer_dict :

    if '_' in Answer_one :
        start1 = time.time()
        name = Answer_one.split("_")
        if name[0] in processedList :
            continue
        preprocessedList = []
        c = 0
        while True :
            pname = name[0]+"_"+str(c)
            if pname in Answer_dict :            
                preprocessedList.append(pname)
                getRaw(pname)
                c += 1
            else :
                break
        end1 = time.time()
        savetime1 += end1 - start1
        processedList.append(name[0])
        flag = True
        while flag :
            flag = False
            pairs =list(itertools.combinations(preprocessedList, 2))
            
            for pair in pairs:
                pair = list(pair)

                raws1 = Answer_dict[pair[0]]['raws']
                raws2 = Answer_dict[pair[1]]['raws']
                
                for ra1, ra2 in zip(raws1, raws2):
                    site1 = ra1['site']
                    site2 = ra2['site']
                    inst1 = Answer_dict[pair[0]][site1]['oriInst']
                    inst2 = Answer_dict[pair[1]][site2]['oriInst']

                    if Secondary_filter(name[0], site1, inst1, ra1, site2, inst2, ra2) >= 3:
                        deleteList.append(pair[1])
                        for site_one in site:
                            if site_one in Answer_dict[pair[1]]:
                                if site_one in Answer_dict[pair[0]].keys() :                            
                                    Answer_dict[pair[0]][site_one]['A_id'].extend(Answer_dict[pair[1]][site_one]['A_id'])
                                    Answer_dict[pair[0]][site_one]['papers'].extend(Answer_dict[pair[1]][site_one]['papers'])
                                    Answer_dict[pair[0]]['raws'].extend(Answer_dict[pair[1]]['raws'])
                                else:
                                    Answer_dict[pair[0]][site_one] = Answer_dict[pair[1]][site_one]
                        flag = True
                        preprocessedList.remove(pair[1])
                        break
                if flag :
                    break
        end2 = time.time()
        savetime2 += end2 - end1    
for d in deleteList:
    del Answer_dict[d]

for d in Answer_dict : 
    if 'raws' in Answer_dict[d] :
        del Answer_dict[d]['raws']
# print(savetime1, savetime2)
# print(Answer_dict)
# id_domestic.insert_many(Answer_dict.values()) #mongodb 추가

1.420976161956787 0.029595613479614258


In [ ]:
analyzer = multicpu.run_factor_integration(keyid, f_id)
analyzer.run()